In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [16]:
df = pd.read_csv('../data/train_data_encoded.csv')
print('shape:', df.shape)
df.head(5)

shape: (28942, 7)


,weight,quality,color,clarity,table_percent,price,depth_percent
0,0.30,3,6,1,0.58,6.353,0.624
1,1.01,4,5,5,0.56,9.183,0.627
2,0.72,4,4,3,0.59,7.983,0.618
3,1.08,2,3,1,0.57,8.371,0.632
4,0.36,3,3,4,0.59,6.588,0.623


In [18]:
#reduce predictor variables
df = df[['weight', 'quality', 'color', 'clarity', 'price']]
df

,weight,quality,color,clarity,price
0,0.30,3,6,1,6.353
1,1.01,4,5,5,9.183
2,0.72,4,4,3,7.983
3,1.08,2,3,1,8.371
4,0.36,3,3,4,6.588
...,...,...,...,...,...
28937,1.52,3,6,1,9.093
28938,0.37,3,4,3,6.545
28939,1.01,3,4,4,8.854
28940,0.80,1,3,1,7.768


In [19]:
#split dara train and test
X = df.drop(["price"], axis = 1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size   = 0.8, random_state = 42)

print("Datos de entrenamiento")
print("-----------------------")
print(y_train.describe())
print('\n')
print("Datos de testeo")
print("-----------------------")
print(y_test.describe())

Datos de entrenamiento
-----------------------
count    23153.000000
mean         7.992375
std          0.993806
min          5.787000
25%          7.156000
50%          8.082000
75%          8.755000
max          9.842000
Name: price, dtype: float64


Datos de testeo
-----------------------
count    5789.000000
mean        8.006556
std         0.986775
min         5.866000
25%         7.189000
50%         8.089000
75%         8.755000
max         9.841000
Name: price, dtype: float64


In [20]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    resultados = {'MAE': [metrics.mean_absolute_error(y_test, y_test_pred), metrics.mean_absolute_error(y_train, y_train_pred)],
                'MSE': [metrics.mean_squared_error(y_test, y_test_pred), metrics.mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)), np.sqrt(metrics.mean_squared_error(y_train, y_train_pred))],
                'R2':  [metrics.r2_score(y_test, y_test_pred), metrics.r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [21]:
param = {"max_depth": [12, 14],
        "max_features": [5, 8],
        "min_samples_split": [15, 30]}

random_forest = GridSearchCV(
            estimator=RandomForestRegressor(),
            param_grid= param,
            cv=10,
            n_jobs = -1,
            verbose= 1,
            return_train_score = True,
            scoring="neg_mean_squared_error")

random_forest.fit(X_train, y_train)

best_rf = random_forest.best_estimator_
print(best_rf)

y_pred_test_rf = random_forest.predict(X_test)
y_pred_train_rf = random_forest.predict(X_train)
rf_results_1 = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Random Forest 1")
rf_results_1.style.background_gradient(cmap='coolwarm')

Fitting 10 folds for each of 8 candidates, totalling 80 fits
RandomForestRegressor(max_depth=12, max_features=5, min_samples_split=15)


,MAE,MSE,RMSE,R2,set,modelo
0,0.080570,0.011139,0.105541,0.988559,test,Random Forest 1
1,0.071835,0.008907,0.094379,0.990981,train,Random Forest 1


In [22]:
param = {"max_depth": [8, 12],
        "max_features": [3, 6],
        "min_samples_split": [10, 20]}

random_forest = GridSearchCV(
            estimator=RandomForestRegressor(),
            param_grid= param,
            cv=10,
            n_jobs = -1,
            verbose= 1,
            return_train_score = True,
            scoring="neg_mean_squared_error")

random_forest.fit(X_train, y_train)

best_rf = random_forest.best_estimator_
print(best_rf)

y_pred_test_rf = random_forest.predict(X_test)
y_pred_train_rf = random_forest.predict(X_train)
rf_results_2 = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Random Forest 2")
rf_results_2.style.background_gradient(cmap='coolwarm')

Fitting 10 folds for each of 8 candidates, totalling 80 fits
RandomForestRegressor(max_depth=12, max_features=3, min_samples_split=10)


,MAE,MSE,RMSE,R2,set,modelo
0,0.079945,0.010987,0.104820,0.988714,test,Random Forest 2
1,0.070634,0.008659,0.093054,0.991232,train,Random Forest 2


In [23]:
param = {"max_depth": [12, 14],
        "max_features": [2, 4],
        "min_samples_split": [10, 15]}

random_forest = GridSearchCV(
            estimator=RandomForestRegressor(),
            param_grid= param,
            cv=10,
            n_jobs = -1,
            verbose= 1,
            return_train_score = True,
            scoring="neg_mean_squared_error")

random_forest.fit(X_train, y_train)

best_rf = random_forest.best_estimator_
print(best_rf)

y_pred_test_rf = random_forest.predict(X_test)
y_pred_train_rf = random_forest.predict(X_train)
rf_results_3 = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Random Forest 3")
rf_results_3.style.background_gradient(cmap='coolwarm')

Fitting 10 folds for each of 8 candidates, totalling 80 fits
RandomForestRegressor(max_depth=12, max_features=2, min_samples_split=10)


,MAE,MSE,RMSE,R2,set,modelo
0,0.080200,0.011066,0.105195,0.988634,test,Random Forest 3
1,0.071435,0.008838,0.094013,0.991051,train,Random Forest 3


In [24]:
df_results = pd.concat([rf_results_1, rf_results_2, rf_results_3], axis = 0)
df_results

,MAE,MSE,RMSE,R2,set,modelo
0,0.080570,0.011139,0.105541,0.988559,test,Random Forest 1
1,0.071835,0.008907,0.094379,0.990981,train,Random Forest 1
0,0.079945,0.010987,0.104820,0.988714,test,Random Forest 2
1,0.070634,0.008659,0.093054,0.991232,train,Random Forest 2
0,0.080200,0.011066,0.105195,0.988634,test,Random Forest 3
1,0.071435,0.008838,0.094013,0.991051,train,Random Forest 3
